### Import required toolboxes and create dropdown list for dataset selection.

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jun 15 13:28:31 2018

@author: rthomas
"""
!pip install ipywidgets
import datetime
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import os
import pandas as pd

dtype = widgets.Dropdown(
    options=['wave_spectral', 'wave_zero', 'weather'],
    disabled=False,
)
print("Select dataset to use:")
display(dtype)

You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'ipywidgets'

### Run the following cell to download the selected dataset from the Marine Institute ERDDAP instance:

In [ ]:
typed = dtype.value
print(typed)

# Set ERDDAP server details
s = 'https://erddap.marine.ie/erddap'
p = 'tabledap'
r = 'csv'

# Set global variables for the parameters without 'buoy_id', 'latitude' and 'longitude' in the list
now = datetime.date.today()
now_string = now.strftime('%Y-%m-%d')

metadata = ['station_id',
            'time']

# Set variables based on data type (typed)
if typed.lower() == 'wave_spectral':
    dataset_id = 'IWaveBNetwork_spectral'
    syear = 2008    
    master_params = ['PeakPeriod',
                     'PeakDirection',
                     'PeakSpread',
                     'SignificantWaveHeight',
                     'EnergyPeriod',
                     'MeanWavePeriod_Tm01',
                     'MeanWavePeriod_Tm02',
                     'qcflag']

elif typed.lower() == 'wave_zero':
    dataset_id = 'IWaveBNetwork_zerocrossing'
    syear = 2008    
    master_params = ['Hmax',
                     'HmaxPeriod',
                     'Havg',
                     'Tavg',
                     'qcflag']

elif typed.lower() == 'weather':
    dataset_id = 'IWBNetwork'
    syear = 2001
    master_params = ['AtmosphericPressure',
                     'WindDirection',
                     'WindSpeed',
                     'Gust',
                     'WaveHeight',
                     'WavePeriod',
                     'MeanWaveDirection',
                     'Hmax',
                     'AirTemperature',
                     'DewPoint',
                     'SeaTemperature',
                     'RelativeHumidity',
                     'QC_Flag']

# Generate parameter component of URL
plist = ''
for item in metadata + master_params:
    plist = plist+item+'%2C'
plist = plist[0:-3]

# Create dataframe for population
df = pd.DataFrame()

# Iterate by year to reduce risk of time out on large time-series
years = range(syear,now.year)
for year in years:    
    url = s+"/"+p+"/"+dataset_id+"."+r+"?"+plist+"&time%3E="+str(year)+"-01-01T00:00:00Z&time%3C"+str(year+1)+"-01-01T00:00:00Z"
    dfbyyear = pd.read_csv(url,index_col=1,header=[0],skiprows=[1],parse_dates=True,infer_datetime_format=True)
    df = pd.concat([df,dfbyyear])
    print("Downloaded %s" % (year))

# Final call for data from start of current year upto midnight of the current day
url = s+"/"+p+"/"+dataset_id+"."+r+"?"+plist+"&time%3E="+str(now.year)+"-01-01T00:00:00Z&time%3C"+now_string+"T00:00:00Z"
dfbyyear = pd.read_csv(url,index_col=1,header=[0],skiprows=[1],parse_dates=True,infer_datetime_format=True)
df = pd.concat([df,dfbyyear])
print("Downloaded %s" % (str(now.year)))

# Make a copy of the unaltered data download
data_full = df.copy()
print("Full resolution data downloaded. Available as 'data_full'.")


### Run the following cell to see full resolution data dataframe structure:

In [ ]:
data_full.head()

### Calculate data availability as a percentage of expected data per day for each variable:

In [ ]:
# Utilise quality control flags to clean data set
# Code to be added...



# Take a working copy of the downloaded data
data = df.copy()

# Add columns for date variable
data['Date'] = data.index.date

# Get a count of data points grouped by station and date
data_summ = data.groupby(['station_id','Date']).count().reset_index(level=['station_id','Date'])

# Create master availability dataframe to hold count converted to percentage of expected data points per day
data_avail = pd.DataFrame()  

# Loop through each station due to different data resolutions and calculate percentages
for stn in data_summ.station_id.unique().tolist():
    data_stn = data_summ[data_summ['station_id']==stn].copy()

    # Set the expected number of data points per day for a buoy type or station
    if typed == 'weather' or stn == 'Westwave MK4':
        res=24
    else:
        res=48

    # Convert counts to percentage
    data_stn.loc[:,master_params] = data_stn.loc[:,master_params]/res*100
    

    # Expand date range to cover full months.
    # Enables accurate calculation of monthly perentage return from the daily data when plotting
    data_fulldates = pd.DataFrame(index = pd.date_range(data_stn.Date.min() - datetime.timedelta(days=data_stn.Date.min().day-1), 
                                                      data_stn.Date.max()))

    # Add date factors to faciliate plotting
    data_fulldates['Date'] = data_fulldates.index.date
    data_fulldates['Year'] = data_fulldates.index.year
    data_fulldates['Month'] = data_fulldates.index.month
    data_fulldates['DOY'] = data_fulldates.index.dayofyear
    
    # Merge individual station dataframe into master availability dataframe and fill blanks dates with zero
    data_fulldates = data_fulldates.merge(data_stn, how='outer', left_on='Date', right_on='Date').fillna(0)
    # Set station
    data_fulldates.loc[:,'station_id'] = stn
    # Add data for the station to the master availability data frame
    data_avail = pd.concat([data_avail,data_fulldates])

# Set indices and tidy up dataframe
data_avail = data_avail.set_index(['station_id', 'Date','Year','Month','DOY'])

if typed != 'weather':
    qc = 'qcflag'
else:
    qc = 'QC_Flag'

data_avail = data_avail.drop([qc], axis=1)
data_avail.columns = pd.MultiIndex.from_product([data_avail.columns, ['avail']])

print("Daily availability generated. Available as 'data_avail'.")

### Run the following cell to see data availability dataframe structure:

In [ ]:
data_avail.head()


### Calculate daily summary statistics

In [ ]:
# Split out parameter types for different summary statistics

params = []
param_dir = []
for item in master_params:
    if 'qc' not in item.lower():
        if 'Dir' in item:
            param_dir.append(item)
        else:
            params.append(item)
#%% Take a copy of the data
data = df.copy()
data['Date'] = data.index.date
data['Year'] = data.index.year

# Get north and east components for directional measurements
param_comp = []
for dirtn in param_dir:
    data['%s_n' % (dirtn)] = np.cos(data[dirtn]*np.pi/180)
    param_comp.append('%s_n' % (dirtn))
    data['%s_e' % (dirtn)] = np.sin(data[dirtn]*np.pi/180)
    param_comp.append('%s_e' % (dirtn))

# Resample for summary statistics for non-directional measurements
daily = data.groupby(['station_id','Date','Year'])[params].agg(['min','max','mean','std'])

if len(param_dir)!=0:
    # Resample for mean and std for directional measurement components (north and east)
    data2 = data.groupby(['station_id','Date','Year'])[param_comp].agg(['mean','std'])

    # Recalculate direction mean and std from averaged components (north and east)
    # Add directly into daily dataframe
    for dirtn in param_dir:
        daily[(dirtn, 'mean')] = (360 + np.arctan2(data2[('%s_e' % (dirtn), 'mean')], data2[('%s_n' % (dirtn), 'mean')]) * 180/np.pi) % 360
        daily[(dirtn, 'std')] = (360 + np.arctan2(data2[('%s_e' % (dirtn), 'std')], data2[('%s_n' % (dirtn), 'std')]) * 180/np.pi) % 360
        daily[(dirtn, 'max')] = np.nan
        daily[(dirtn, 'min')] = np.nan

# Sort daily dataframe
daily = daily[sorted(daily.columns.tolist())]
data_daily = daily
print("Daily statistics generated. Available as 'data_daily'.")

### Run the following cell to see the daily summary statistics dataframe structure:

In [ ]:
data_daily.head()

### Plot data with interactive widgets:

In [ ]:
from ipywidgets import interact
import matplotlib.pyplot as plt
import seaborn as sns

plot_daily = data_daily.loc[data_daily.index.levels[0].tolist()[0]]

def plotting(station, year, xaxis, yaxis, stat):
    idx = pd.IndexSlice

    plot_daily = data_daily.loc[station]
    
    if xaxis in ('PeakDirection','WindDirection','MeanWaveDirection'):
        statx = 'mean'
    else:
        statx = stat
    if yaxis in ('PeakDirection','WindDirection','MeanWaveDirection'):
        staty = 'mean'
    else:
        staty = stat
        
    x = plot_daily.loc[idx[:,year],idx[[xaxis],statx]].values
    y = plot_daily.loc[idx[:,year],idx[[yaxis],staty]].values

    if xaxis in ('PeakDirection','WindDirection','MeanWaveDirection'):
        ax = plt.subplot(111, projection='polar')
        ax.set_theta_direction(-1)      # Set degrees to match compass rose
        ax.set_theta_zero_location("N") # Set zero to north on the plot
        ax.set_rlabel_position(180)     # Set labels to area on plot with minimal data
        ax = plt.scatter(x*np.pi/180, y)
        plt.title("Location: %s Year: %s" % (station,year))
        plt.xlabel("%s (%s) [unit]" % (yaxis, staty))

    else:
        l = plt.plot(x, y,'o')
        plt.setp(l, markersize=5)
        plt.setp(l, markerfacecolor='C0')
        plt.title("Location: %s Year: %s" % (station,year))
        plt.xlabel("%s (%s) [unit]" % (xaxis, statx))
        plt.ylabel("%s (%s) [unit]" % (yaxis, staty))
        
    plt.show()

interact(plotting,
    station = widgets.Dropdown(
        options=data_daily.index.levels[0].tolist(),
        description='Location:',
        disabled=False),

    year = widgets.IntSlider(
        min = int(plot_daily.index.levels[1].min()),
        max = int(plot_daily.index.levels[1].max()),
        description = 'Year:'),

    xaxis = widgets.Dropdown(
        options=data_daily.columns.levels[0].tolist(),
        description='x-axis:',
        disabled=False),

    yaxis = widgets.Dropdown(
        options=data_daily.columns.levels[0].tolist(),
        description='y-axis:',
        disabled=False),

    stat = widgets.RadioButtons(
        options=data_daily.columns.levels[1].tolist(),
        description='Statistic:',
        disabled=False),
        )

def avail_map(station1,parameter1,interval):
    plot_avail = data_avail.loc[station1]
    plot_avail.columns = plot_avail.columns.droplevel(level=1)
    
    if interval == 'Day':
        plot_avail_set = plot_avail
        ycol = 'DOY'
        tag=False
    elif interval == 'Month':
        plot_avail_set = plot_avail.groupby(['Year','Month'])[data_avail.columns.levels[0].tolist()].agg(['mean'])
        ycol='Month'
        tag=True
        plot_avail_set.columns = plot_avail_set.columns.droplevel(level=1)

    plot_avail_set = plot_avail_set.reset_index()
    pivot = plot_avail_set.pivot(index='Year', columns=ycol, values=parameter1)
    ax = sns.heatmap(pivot,annot=tag)
    plt.title("%s data availability (as percentage per %s): %s" % (station1, interval, parameter1))
    plt.show()
    
interact(avail_map,    
    station1 = widgets.Dropdown(
        options=data_avail.index.levels[0].tolist(),
        description='Location:',
        disabled=False),
         
    parameter1 = widgets.Dropdown(
        options=data_avail.columns.levels[0].tolist(),
        description='Parameter:',
        disabled=False),
    
    interval = widgets.RadioButtons(
        options=['Month','Day'],
        description='Interval:',
        disabled=False),
        )
        

### Commands to save data as csv files. 

Will not work from MyBinder container.

Input details of the working directory to save the csv files and uncomment the commands (remove the # from the start of each line) to make the cells run.

In [ ]:
#working_directory = os.path.normpath('C:/Downloads')

In [ ]:
#datafile = os.path.join(working,'%s_data_raw.csv' % (typed, typed))
#availfile = os.path.join(working,'%s_data_availability.csv' % (typed, typed))
#dailyfile = os.path.join(working,'%s_daily_summary.csv' % (typed, typed))

#data_full.to_csv(datafile, sep=',')
#data_avail.to_csv(availfile, sep=',')
#data_daily.to_csv(dailyfile, sep=',')

In [ ]:
data_full.head()

### Run following cell to calculate Power percentiles for each site and overlay percentile lines on wave height period scatter plots. 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

for stn in data_full['station_id'].unique().tolist():
    stn_data = data_full[data_full['station_id']==stn].loc[:,['SignificantWaveHeight','EnergyPeriod']]

    stn_data['P'] = 0.49 * stn_data.loc[:,'SignificantWaveHeight']**2 * stn_data.loc[:,'EnergyPeriod']

    P_mean = stn_data.loc[:,'P'].mean()
    P_max = stn_data.loc[:,'P'].max()
    P_min = stn_data.loc[:,'P'].min()
    P_90 = np.percentile(stn_data.loc[:,'P'].values,90)
    P_99 = np.percentile(stn_data.loc[:,'P'].values,99)

    print("%s: Power - mean = %s kW/m" % (stn, P_mean))
    print("%s: Power - max = %s kW/m" % (stn, P_max))
    print("%s: Power - min = %s kW/m" % (stn, P_min))
    print("%s: Power - 90th percentile = %s kW/m" % (stn, P_90))
    print("%s: Power - 99th percentile = %s kW/m" % (stn, P_99))

    x = stn_data.loc[:,'EnergyPeriod'].values
    y = stn_data.loc[:,'SignificantWaveHeight'].values
    x99 = range(1,26)
    y99 = []
    y90 = []
    for i in x99:
        y99.append(P_99/(0.49*i))
        y90.append(P_90/(0.49*i))

    l = plt.plot(x, y,'o')
    plt.setp(l, markersize=5)
    plt.setp(l, markerfacecolor='C0')
    m = plt.plot(x99, y99,linestyle='solid')
    plt.setp(m, markersize=10)
    plt.setp(m, color='red')
    m = plt.plot(x99, y90,linestyle='solid')
    plt.setp(m, markersize=10)
    plt.setp(m, color='green')
    plt.axis([0, 25, 0, 25])
    plt.title("Location: %s" % (stn))
    plt.xlabel("Energy Period [s]" )
    plt.ylabel("Significant Wave Hieght [m]" )

    plt.show()